In [68]:
import re
import time

from peewee import SqliteDatabase, Model, TextField


db = SqliteDatabase('places.sqlite')
        
class Place(Model):
    name_and_description = TextField()
    tags = TextField(null=True)
    
    class Meta:
        database = db

db.create_tables([Place])

In [69]:
from pathlib import Path


def places_from_txt_to_db(txt_filename):
    # places are listed in the following format
    # 1. **Café du Monde**
    #    *Description:* Lorem ipsum.
    # 
    # 2. **Central Park**
    #    *Description:* Foobar baz.
    #
    # 3. **The Louvre Museum**
    #    *Description:* Bla bal bla.

    text = Path(txt_filename).read_text()
    places_with_description = text.split("\n\n")
    for place in places_with_description:
        # remove formatting (*) and digits
        formatted_place = place.replace("*", "")
        formatted_place = re.sub("\d+\.\s?", "", formatted_place)
        # save to db
        Place.get_or_create(name_and_description=formatted_place)

In [70]:
places_from_txt_to_db("gpt_places.txt")

In [71]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

def next_message(dialog):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=dialog
    )

    return response["choices"][0]["message"]

In [81]:
def tag_batch(places):
    
    prompt = f"""You are an AI expert. Please analyze a list of {len(places)} locations below. Please create 20-30 tags for each location. Tags may include but not limited to location name, location city and country if specified, general information about location, emotions associated with location. You can also include activities and items related to the location.
    
     
    For example, for a cafe in Paris the could be: cafe, paris, france, breakfast, dinner, lunch, fulfilled, cheerful, relax, food, table, chair, soup, coffee, wine, social, etc.
     
    The algorithm used to compare tags is levenshtein_distance. Please optimize tags for this task. Please split tags ONLY by commas. 

    Please respond in a following format:
    
    Place 0: tag1, tag2, tag3
    Place 1: tag1, tag2, tag3
    ...
    
    Below are the places with descriptions.
    
    {list(enumerate(map(lambda p: p.name_and_description, places)))}"""
    
    result = next_message([
        {"role": "system", "content": "You are an AI expert."},
        {"role": "user", "content": prompt},
    ])
    
    all_places_text = result["content"]
    probably_tag_lines = [i for i in all_places_text.split("\n") if i.strip()]
    
    for i, single_place_text in enumerate(probably_tag_lines):
        tags = single_place_text.split(":", maxsplit=1)[-1].strip()
        place = places[i]
        place.tags = tags
        place.save()


def tag_untagged_places(batch_size=10):
    untagged_places = list(Place.select().where((Place.tags == None) | (Place.tags == '')))
    
    for i in range(0, len(untagged_places), batch_size):
        print(f"Creating tags {i+1}-{i+batch_size} of {len(untagged_places)}...", end=" ")
        places_to_tag = untagged_places[i:i+batch_size]
        tag_batch(places_to_tag)
        print("Done!")
        # if using a free key, this helps to getting rate limited
        time.sleep(60)

In [84]:
tag_untagged_places()

Creating tags 1-10 of 61... Done!
Creating tags 11-20 of 61... Done!
Creating tags 21-30 of 61... Done!
Creating tags 31-40 of 61... Done!
Creating tags 41-50 of 61... Done!
Creating tags 51-60 of 61... Done!
Creating tags 61-70 of 61... Done!


In [112]:
def get_question_tags(question):
    
    prompt = f"""You are an AI expert. Please analyze user's question below. Please create 5-7 tags. Tags may include but not limited to probable locations names, locations cities and countries if specified, general information about the question, emotions associated with the question. You can also include activities and items related to the question.
    
    For example, here are possible for a question "I'd like to swim": sea, pool, river, beach, activities, happy, sport, sand, water, swim, swimsuit, diving
     
    The algorithm used to compare tags is levenshtein_distance. Please optimize tags for this task. Please split tags ONLY by commas. 

    Please respond in a following format:
    
    tag1, tag2, tag3
    
    Below is the user's question.
    
    "{question}"""
    
    result = next_message([
        {"role": "system", "content": "You are an AI expert."},
        {"role": "user", "content": prompt},
    ])
    
    content = result["content"]
    # remove first phrase like "Here are the tags:"
    return content.lower().split(":", maxsplit=1)[-1]

In [104]:
get_question_tags("I want some coffee")

'coffee, beverage, drink, caffeine, morning, wake-up, hot, aroma, coffee shop, espresso, cappuccino'

In [120]:
import jellyfish

def sort_places_by_relevance(places, search_tags_text):
    search_tags = [s.strip() for s in search_tags_text.split(",")]
    
    def score(place):
        place_tags = [t.strip() for t in place.tags.lower().split(",")]
        scores = []
        for s in search_tags:
            scores += [jellyfish.levenshtein_distance(s, t) for t in place_tags]
        
        # less distance = closer match
        return sum(sorted(scores)[:len(search_tags)])
    
    return sorted(places, key=score)

⬆️looks for closest matching tag and uses its score in sorting. 

For example, if the question has tags `exteme, sport` and only the best matching tag is used, both places `Dad's gym: sport, running, exercise` and `Everest: mountain everest, dangerous, extreme, sport` will have the save score even though Everest has two mathing tags

In [121]:
def answer_question(question):
    question_tags = get_question_tags(question)
    print("Question tags:", question_tags)
    
    all_places = list(Place.select())
    places_by_relevance = sort_places_by_relevance(all_places, question_tags)
    top_10_places = places_by_relevance[:10]
    
    top_10_places_text = "\n=====\n".join(map(lambda p: p.name_and_description, top_10_places))
    
    prompt = f"""You are places to visit expert. Below is your database of places. Please carefully read the user's question and answer it. You can include any information regarding places described below in your answer. Please recommend to the user places relevant to the question.
    
    Below are the places with descriptions separated by "=====".
    
    {top_10_places_text}"""
    
    result = next_message([
        {"role": "system", "content": prompt},
        {"role": "user", "content": question},
    ])
    
    return result["content"]

In [124]:
answer_question("I want some coffee")

Question tags: coffee, beverage, caffeine


"If you're looking for a coffee fix, I would recommend visiting Café du Monde in the French Quarter of New Orleans. This iconic coffee shop is famous for its beignets and chicory coffee. You can enjoy their delicious pastries while sipping on a cup of coffee and soaking in the open-air ambiance. Another option would be to visit Piazza San Marco in Venice, Italy. This beautiful square is renowned for its stunning architecture and is home to several local cafes where you can enjoy a cup of coffee while people-watching."

In [125]:
answer_question("List 5 meme places")

Question tags: meme places, locations, humorous, internet, popular


"1. The Sistine Chapel, Vatican City\n   Description: Memes related to the Sistine Chapel often humorously speculate on the real backstory behind the creation of Michelangelo's famous paintings.\n\n2. Cocos Island, Costa Rica\n   Description: The presence of sharks in Cocos Island has inspired memes that play on the idea of danger and thrill associated with diving in this popular destination.\n\n3. Mykonos, Greece\n   Description: Mykonos is known for its vibrant nightlife, which has been the subject of humorous memes that capture the energetic party atmosphere of the island.\n\n4. Fiji\n   Description: Fiji's picturesque blue lagoons and stunning natural beauty have been featured in various memes that highlight its status as a tranquil and idyllic tropical getaway.\n\n5. Bali, Indonesia\n   Description: Bali's lush landscapes and unique cultural experiences have inspired memes that humorously depict the island as a paradise for surfers, yoga enthusiasts, and travelers seeking relaxati

In [126]:
answer_question("I'm a danger traveller")

Question tags: travel, adventure, exploration, wanderlust, excitement


'If you\'re an extreme traveler seeking thrilling and adventurous experiences, here are a few recommendations:\n\n1. The Running of the Bulls, Pamplona, Spain: Join the adrenaline-pumping tradition of running with the bulls during the San Fermín festival. It\'s a daring and unique experience that tests your courage and agility.\n\n2. The Great Blue Hole, Belize: Dive or snorkel in the Great Blue Hole, a massive underwater sinkhole known for its eerie, deep blue color. Explore its depths and witness the diverse marine life that calls this natural wonder home.\n\n3. The Door to Hell, Turkmenistan: Visit the Darvaza Gas Crater, also known as the "Door to Hell." This burning natural gas pit in the Karakum Desert offers a surreal and mesmerizing sight, especially at night.\n\n4. Zion National Park, USA: If you love outdoor adventures, Zion National Park is a must-visit. This Utah treasure offers thrilling hiking trails, canyoneering opportunities, and breathtaking views of towering red rock